In [1]:
import geopandas as gpd
from pyspatialml import Raster
from copy import deepcopy
import os
import tempfile
import rasterio.plot
import numpy as np
from joblib import load
import glob
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

In [ ]:
#%% load model

model_path = 'codici/model/gb_29042024.joblib'
model = load(model_path)

#%% load predictors and stack
# Define the folder containing the raster files
folder_path = 'dati/predictors'
# Define the pattern to match all files with the .tif extension
file_pattern = "*.tif"
# Combine the folder path and file pattern to search for files
search_pattern = os.path.join(folder_path, file_pattern)

# Use glob.glob to find all files matching the pattern
predictors = glob.glob(search_pattern)

stack = Raster(predictors)

In [ ]:
#%% Prediction
result = stack.predict(estimator=model, dtype='int16', nodata=-999)
result_probs = stack.predict_proba(estimator=model)

In [ ]:
#%% plot classification result
result.iloc[0].cmap = "terrain"
result.iloc[0].categorical = True
result.plot()
plt.show()

#%% Plot Predicted Probabilities
result_probs.prob_1.cmap = 'rainbow'

mpl.style.use('fast')

result_probs.iloc[1].plot(out_shape=(200, 200), figsize=(10, 8))
plt.show()

In [ ]:
#%% save predictions
result.write("output/ls_pred.tif")
result_probs.write("output/ls_pred_proba.tif")